In [4]:
def makeSuffixArrayByInducedSorting(string, alphabetSize):
 """
 Compute the suffix array of 'string' with the SA-IS algorithm.
 """
 # Classify each character of the string as S_TYPE or L_TYPE
 typemap = buildTypeMap(string)

 # We'll be slotting suffixes into buckets according to what
 # character they start with, so let's precompute that info now.
 bucketSizes = findBucketSizes(string, alphabetSize)

 # Use a simple bucket-sort to insert all the LMS suffixes into
 # approximately the right place the suffix array.
 guessedSuffixArray = guessLMSSort(string, bucketSizes, typemap)

 # Slot all the other suffixes into guessedSuffixArray, by using
 # induced sorting. This may move the LMS suffixes around.
 induceSortL(string, guessedSuffixArray, bucketSizes, typemap)
 induceSortS(string, guessedSuffixArray, bucketSizes, typemap)

 # Create a new string that summarises the relative order of LMS
 # suffixes in the guessed suffix array.
 summaryString, summaryAlphabetSize, summarySuffixOffsets = \
     summariseSuffixArray(string, guessedSuffixArray, typemap)

 # Make a sorted suffix array of the summary string.
 summarySuffixArray = makeSummarySuffixArray(
     summaryString,
     summaryAlphabetSize,
 )

 # Using the suffix array of the summary string, determine exactly
 # where the LMS suffixes should go in our final array.
 result = accurateLMSSort(string, bucketSizes, typemap,
         summarySuffixArray, summarySuffixOffsets)

 # ...and once again, slot all the other suffixes into place with
 # induced sorting.
 induceSortL(string, result, bucketSizes, typemap)
 induceSortS(string, result, bucketSizes, typemap)

 return result

In [5]:
S_TYPE = ord("S")
L_TYPE = ord("L")

In [6]:
def buildTypeMap(data):
     """
     Builds a map marking each suffix of the data as S_TYPE or L_TYPE.
     """
     # The map should contain one more entry than there are characters
     # in the string, because we also need to store the type of the
     # empty suffix between the last character and the end of the
     # string.
     res = bytearray(len(data) + 1)

     # The empty suffix after the last character is S_TYPE
     res[-1] = S_TYPE

     # If this is an empty string...
     if not len(data):
         # ...there are no more characters, so we're done.
         return res

     # The suffix containing only the last character must necessarily
     # be larger than the empty suffix.
     res[-2] = L_TYPE

     # Step through the rest of the string from right to left.
     for i in range(len(data)-2, -1, -1):
         if data[i] > data[i+1]:
             res[i] = L_TYPE
         elif data[i] == data[i+1] and res[i+1] == L_TYPE:
             res[i] = L_TYPE
         else:
             res[i] = S_TYPE

     return res

In [7]:
def lmsSubstringsAreEqual(string, typemap, offsetA, offsetB):
     """
     Return True if LMS substrings at offsetA and offsetB are equal.
     """
     # No other substring is equal to the empty suffix.
     if offsetA == len(string) or offsetB == len(string):
         return False

     i = 0
     while True:
         aIsLMS = isLMSChar(i + offsetA, typemap)
         bIsLMS = isLMSChar(i + offsetB, typemap)

         # If we've found the start of the next LMS substrings...
         if (i > 0 and aIsLMS and bIsLMS):
             # ...then we made it all the way through our original LMS
             # substrings without finding a difference, so we can go
             # home now.
             return True

         if aIsLMS != bIsLMS:
             # We found the end of one LMS substring before we reached
             # the end of the other.
             return False

         if string[i + offsetA] != string[i + offsetB]:
             # We found a character difference, we're done.
             return False

         i += 1

In [8]:
def findBucketSizes(string, alphabetSize=256):
     res = [0] * alphabetSize

     for char in string:
         res[char] += 1

     return res

In [9]:
>>> def findBucketTails(bucketSizes):
     offset = 1
     res = []
     for size in bucketSizes:
         offset += size
         res.append(offset - 1)

     return res

In [10]:
def findBucketHeads(bucketSizes):
     offset = 1
     res = []
     for size in bucketSizes:
         res.append(offset)
         offset += size

     return res

In [11]:
 def isLMSChar(offset, typemap):
     """
     Returns true if the character at offset is a left-most S-type.
     """
     if offset == 0:
         return False
     if typemap[offset] == S_TYPE and typemap[offset - 1] == L_TYPE:
         return True
 
     return False

In [12]:
def showSuffixArray(arr, pos=None):
    i=0
    i=i+1

In [13]:
def guessLMSSort(string, bucketSizes, typemap):
     """
     Make a suffix array with LMS-substrings approximately right.
     """
     # Create a suffix array with room for a pointer to every suffix of
     # the string, including the empty suffix at the end.
     guessedSuffixArray = [-1] * (len(string) + 1)

     bucketTails = findBucketTails(bucketSizes)

     # Bucket-sort all the LMS suffixes into their appropriate bucket.
     for i in range(len(string)):
         if not isLMSChar(i, typemap):
             # Not the start of an LMS suffix
             continue

         # Which bucket does this suffix go into?
         bucketIndex = string[i]
         # Add the start position at the tail of the bucket...
         guessedSuffixArray[bucketTails[bucketIndex]] = i
         # ...and move the tail pointer down.
         bucketTails[bucketIndex] -= 1

         # Show the current state of the array
         showSuffixArray(guessedSuffixArray)

     # The empty suffix is defined to be an LMS-substring, and we know
     # it goes at the front.
     guessedSuffixArray[0] = len(string)

     showSuffixArray(guessedSuffixArray)

     return guessedSuffixArray

In [14]:
def induceSortL(string, guessedSuffixArray, bucketSizes, typemap):
     """
     Slot L-type suffixes into place.
     """
     bucketHeads = findBucketHeads(bucketSizes)

     # For each cell in the suffix array....
     for i in range(len(guessedSuffixArray)):
         if guessedSuffixArray[i] == -1:
             # No offset is recorded here.
             continue

         # We're interested in the suffix that begins to the left of
         # the suffix this entry points at.
         j = guessedSuffixArray[i] - 1
         if j < 0:
             # This entry in the suffix array is the suffix that begins
             # at the start of the string, offset 0. Therefore there is
             # no suffix to the left of it, and j is out of bounds of
             # the typemap.
             continue
         if typemap[j] != L_TYPE:
             # We're only interested in L-type suffixes right now.
             continue

         # Which bucket does this suffix go into?
         bucketIndex = string[j]
         # Add the start position at the head of the bucket...
         guessedSuffixArray[bucketHeads[bucketIndex]] = j
         # ...and move the head pointer up.
         bucketHeads[bucketIndex] += 1

         showSuffixArray(guessedSuffixArray, i)

In [15]:
def induceSortS(string, guessedSuffixArray, bucketSizes, typemap):
     """
     Slot S-type suffixes into place.
     """
     bucketTails = findBucketTails(bucketSizes)

     for i in range(len(guessedSuffixArray)-1, -1, -1):
         j = guessedSuffixArray[i] - 1
         if j < 0:
             # This entry in the suffix array is the suffix that begins
             # at the start of the string, offset 0. Therefore there is
             # no suffix to the left of it, and j is out of bounds of
             # the typemap.
             continue
         if typemap[j] != S_TYPE:
             # We're only interested in S-type suffixes right now.
             continue

         # Which bucket does this suffix go into?
         bucketIndex = string[j]
         # Add the start position at the tail of the bucket...
         guessedSuffixArray[bucketTails[bucketIndex]] = j
         # ...and move the tail pointer down.
         bucketTails[bucketIndex] -= 1

         showSuffixArray(guessedSuffixArray, i)

In [16]:
def summariseSuffixArray(string, guessedSuffixArray, typemap):
     """
     Construct a 'summary string' of the positions of LMS-substrings.
     """
     # We will use this array to store the names of LMS substrings in
     # the positions they appear in the original string.
     lmsNames = [-1] * (len(string) + 1)

     # Keep track of what names we've allocated.
     currentName = 0

     # Where in the original string was the last LMS suffix we checked?
     lastLMSSuffixOffset = None

     # We know that the first LMS-substring we'll see will always be
     # the one representing the empty suffix, and it will always be at
     # position 0 of suffixOffset.
     lmsNames[guessedSuffixArray[0]] = currentName
     lastLMSSuffixOffset = guessedSuffixArray[0]

     showSuffixArray(lmsNames)

     # For each suffix in the suffix array...
     for i in range(1, len(guessedSuffixArray)):
         # ...where does this suffix appear in the original string?
         suffixOffset = guessedSuffixArray[i]

         # We only care about LMS suffixes.
         if not isLMSChar(suffixOffset, typemap):
             continue

         # If this LMS suffix starts with a different LMS substring
         # from the last suffix we looked at....
         if not lmsSubstringsAreEqual(string, typemap,
                 lastLMSSuffixOffset, suffixOffset):
             # ...then it gets a new name
             currentName += 1

         # Record the last LMS suffix we looked at.
         lastLMSSuffixOffset = suffixOffset

         # Store the name of this LMS suffix in lmsNames, in the same
         # place this suffix occurs in the original string.
         lmsNames[suffixOffset] = currentName
         showSuffixArray(lmsNames)

     # Now lmsNames contains all the characters of the suffix string in
     # the correct order, but it also contains a lot of unused indexes
     # we don't care about and which we want to remove. We also take
     # this opportunity to build summarySuffixOffsets, which tells
     # us which LMS-suffix each item in the summary string represents.
     # This will be important later.
     summarySuffixOffsets = []
     summaryString = []
     for index, name in enumerate(lmsNames):
         if name == -1:
             continue
         summarySuffixOffsets.append(index)
         summaryString.append(name)

     # The alphabetically smallest character in the summary string
     # is numbered zero, so the total number of characters in our
     # alphabet is one larger than the largest numbered character.
     summaryAlphabetSize = currentName + 1

     return summaryString, summaryAlphabetSize, summarySuffixOffsets

In [17]:
def makeSummarySuffixArray(summaryString, summaryAlphabetSize):
     """
     Construct a sorted suffix array of the summary string.
     """
     if summaryAlphabetSize == len(summaryString):
         # Every character of this summary string appears once and only
         # once, so we can make the suffix array with a bucket sort.
         summarySuffixArray = [-1] * (len(summaryString) + 1)

         # Always include the empty suffix at the beginning.
         summarySuffixArray[0] = len(summaryString)

         for x in range(len(summaryString)):
             y = summaryString[x]
             summarySuffixArray[y+1] = x

     else:
         # This summary string is a little more complex, so we'll have
         # to use recursion.
         summarySuffixArray = makeSuffixArrayByInducedSorting(
             summaryString,
             summaryAlphabetSize,
         )

     return summarySuffixArray

In [18]:
def accurateLMSSort(string, bucketSizes, typemap,
         summarySuffixArray, summarySuffixOffsets):
     """
     Make a suffix array with LMS suffixes exactly right.
     """
     # A suffix for every character, plus the empty suffix.
     suffixOffsets = [-1] * (len(string) + 1)

     # As before, we'll be adding suffixes to the ends of their
     # respective buckets, so to keep them in the right order we'll
     # have to iterate through summarySuffixArray in reverse order.
     bucketTails = findBucketTails(bucketSizes)
     for i in range(len(summarySuffixArray)-1, 1, -1):
         stringIndex = summarySuffixOffsets[summarySuffixArray[i]]

         # Which bucket does this suffix go into?
         bucketIndex = string[stringIndex]
         # Add the suffix at the tail of the bucket...
         suffixOffsets[bucketTails[bucketIndex]] = stringIndex
         # ...and move the tail pointer down.
         bucketTails[bucketIndex] -= 1

         showSuffixArray(suffixOffsets)

     # Always include the empty suffix at the beginning.
     suffixOffsets[0] = len(string)

     showSuffixArray(suffixOffsets)

     return suffixOffsets

In [19]:
def makeSuffixArray(bytestring):
    return makeSuffixArrayByInducedSorting(bytestring, 256)

In [20]:
def downsampleSuffixArray(sa, saFactor):
        return [position for index, position in enumerate(sa) if index % saFactor == 0]

In [21]:
testExamplesDownsampleSuffixArray = [
    { 'input':[6, 5, 2, 3, 0, 4, 1] , 'output': [6, 0] },
    { 'input':[6, 5, 3, 1, 0, 4, 2] , 'output': [6, 0] }
]

for i in range(len(testExamplesDownsampleSuffixArray)):
    assert downsampleSuffixArray(testExamplesDownsampleSuffixArray[i]['input'], 4) == testExamplesDownsampleSuffixArray[i]['output']


In [22]:
def suffixArray(s):
    satups = sorted([(s[i:], i) for i in range(0, len(s))])
    # Extract and return just the offsets
    return map(lambda x: x[1], satups)


In [23]:
testExamplesSuffixArray = [
    { 'input': 'abaaba$', 'output': (6, 5, 2, 3, 0, 4, 1) },
    { 'input': 'banana$', 'output': (6, 5, 3, 1, 0, 4, 2) }
]

for i in range(len(testExamplesSuffixArray)):
   
    assert tuple(suffixArray(testExamplesSuffixArray[i]['input'])) == testExamplesSuffixArray[i]['output']


In [24]:
def bwt(t, sa):
  bw = []
  for si in sa:
     if si == 0:
        bw.append('$')
     else:
        bw.append(t[si-1])
  return ''.join(bw) 


In [25]:
testExamplesBwt = [
    { 'input': 'abaaba$','sa':[6, 5, 2, 3, 0, 4, 1],'output': 'abba$aa' },
    { 'input': 'banana$','sa':[6, 5, 3, 1, 0, 4, 2] ,'output': 'annb$aa' }
]

for i in range(len(testExamplesBwt)):
   
    assert bwt(testExamplesBwt[i]['input'], testExamplesBwt[i]['sa'])  == testExamplesBwt[i]['output']
  

In [26]:
def rankBwt(bw):
 counts = dict() 
 ranks = [] 
 for c in bw:
   if c not in counts:
      counts[c] = 0
   ranks.append(counts[c])
   counts[c] += 1
 return ranks, counts

In [27]:
testExamplesRankBwt = [
    { 'input': 'abba$aa', 'ranks': [0, 0, 1, 1, 0, 2, 3], 'counts':{'a': 4, 'b': 2, '$': 1}},
    { 'input': 'annb$aa', 'ranks': [0, 0, 1, 0, 0, 1, 2], 'counts':{'a': 3, 'n': 2, 'b': 1, '$': 1}}
]

for i in range(len(testExamplesRankBwt)):
    
    ranks, counts = rankBwt(testExamplesRankBwt[i]['input'])
    assert ranks == testExamplesRankBwt[i]['ranks']
    assert counts == testExamplesRankBwt[i]['counts']

In [28]:
def rankAllBwt(bw, tallyFactor):
 """ Given BWT string bw, returns a map of lists. Keys are
 characters and lists are cumulative # of occurrences up to and
 including the row. """
 tots = {}
 rankAll = {}
 for c in bw:
  if c not in tots:
   tots[c] = 0
   rankAll[c] = []
 for index, c in enumerate(bw):
  tots[c] += 1
  for  c in tots.keys():
   if index % tallyFactor == 0:    
     rankAll[c].append(tots[c])
 return rankAll, tots


In [29]:
testExamplesRankAllBwt = [
    { 'input': 'abba$aa', 'ranks': {'a': [1, 1, 1, 2, 2, 3, 4], 'b': [0, 1, 2, 2, 2, 2, 2], '$': [0, 0, 0, 0, 1, 1, 1]}, 'tots':{'a': 4, 'b': 2, '$': 1}}
]
for i in range(len(testExamplesRankAllBwt)):
    ranks, counts = rankAllBwt(testExamplesRankAllBwt[i]['input'],1)
    assert ranks == testExamplesRankAllBwt[i]['ranks']
    assert counts == testExamplesRankAllBwt[i]['tots']
   

In [30]:
def firstCol(counts):
    """ Return a map from characters to the range of cells in the first
 column containing the character. """
    first = {}
    totc = 0
    for c, count in sorted(counts.items()):
           first[c] = (totc, totc + count)
           totc += count
    return first

In [31]:
testExamplesFirstCol = [
    { 'input':{'a': 4, 'b': 2, '$': 1} , 'output': {'$': (0,1), 'a': (1,5), 'b': (5,7)}},
    { 'input': {'a': 3, 'n': 2, 'b': 1, '$': 1}, 'output':{'$': (0,1), 'a': (1,4), 'b': (4,5), 'n': (5,7)}}
]

for i in range(len(testExamplesFirstCol)):
    assert firstCol(testExamplesFirstCol[i]['input']) ==  testExamplesFirstCol[i]['output']
   

In [32]:
def reverseBwt(bw):
    """ Make T from BWT(T) """
    ranks, tots = rankBwt(bw)
    first = firstCol(tots)
    rowi = 0
    t = "$"
    while bw[rowi] != '$':
        c = bw[rowi]
        t = c + t
        rowi = first[c][0] + ranks[rowi]
    return t

In [33]:
testExamplesReverseBwt = [
    { 'input': 'abba$aa', 'output': 'abaaba$' },
    { 'input': 'annb$aa', 'output': 'banana$' }
]

for i in range(len(testExamplesReverseBwt)):
   
    assert reverseBwt(testExamplesReverseBwt[i]['input'])  == testExamplesReverseBwt[i]['output']

In [34]:
def indexSearch(bw, p, sa, ranks, tots):
    """ Given BWT(T) and a pattern string p and suffix array,
 return indexes of appears. """
    if not p:
        return []
    #ranks, tots = rankBwt(bw)
    first = firstCol(tots)
    if p[-1] not in first or p[-1] == '$':
        return []
    l, r = first[p[-1]]
    i = len(p) - 2
    while i >= 0 and r > l:
        c = p[i]
        # scan from left, looking for occurrences of c
        j = l
        while j < r:
            if bw[j] == c:
                l = first[c][0] + ranks[j]
                break
            j += 1
        if j == r:
            l = r
            break  # no occurrences -> no match
        r -= 1
        while bw[r] != c:
            r -= 1
        r = first[c][0] + ranks[r] + 1
        i -= 1
    
    list1 = list(sa)
    return list1[l:r]
    #index = []
    #for i in range(l, r):
        #index.append(list1[i])    
    #return index

In [35]:
testExamplesIndexSearch = [
    { 'bwt':'abba$aa', 'pattern': 'aba', 'sa': (6, 5, 2, 3, 0, 4, 1), 'output': [3, 0] }
]   
   

for i in range(len(testExamplesIndexSearch)): 
    
    assert indexSearch(testExamplesIndexSearch[i]['bwt'], testExamplesIndexSearch[i]['pattern'], testExamplesIndexSearch[i]['sa'],[0, 0, 1, 1, 0, 2, 3], {'a': 4, 'b': 2, '$': 1}) ==  testExamplesIndexSearch[i]['output']

In [36]:
def indexSearchOptimized(bw, p, sa, saFactor, tallyFactor, rankAll, tots):
 """ Given BWT(T) and a pattern string p and suffix array,
 return indexes of appears. """
 if not p:
        return []
 #rankAll, tots = rankAllBwt(bw, tallyFactor)
 first = firstCol(tots)
 if p[-1] not in first or p[-1] == '$':
        return []
 l, r = first[p[-1]]
 i = len(p)-2
 while i >= 0 and r > l:
   c = p[i]
   l = first[c][0] + getTallyRank(c, l-1, tallyFactor, rankAll, bw)
   r = first[c][0] + getTallyRank(c, r-1, tallyFactor, rankAll, bw)
   i -= 1
   
 list1 = list(sa)
 index = []
 for i in range(l, r):
        index.append(position(i, list1, saFactor, bw, first, tallyFactor, rankAll))
      
 return index
 

In [42]:
testExamplesIndexSearchOptimized = [
    { 'bwt':'abba$aa', 'pattern': 'aba', 'sa': (6, 0), 'output': [3, 0]},
    { 'bwt':'annb$aa', 'pattern': 'na', 'sa': (6, 0), 'output': [4, 2]}
]

for i in range(len(testExamplesIndexSearchOptimized)):
    
    assert indexSearchOptimized(testExamplesIndexSearchOptimized[i]['bwt'], testExamplesIndexSearchOptimized[i]['pattern'], testExamplesIndexSearchOptimized[i]['sa'], 4, 2, {'a': [1, 1, 1, 2, 2, 3, 4], 'b': [0, 1, 2, 2, 2, 2, 2], '$': [0, 0, 0, 0, 1, 1, 1]}, {'a': 4, 'b': 2, '$': 1}) ==  testExamplesIndexSearchOptimized[i]['output']

KeyboardInterrupt: 

In [ ]:
 def position(firstColumnIndex, sa, saFactor, bw, first, tallyFactor, tally):
        difference = 0
        while (firstColumnIndex % saFactor != 0):
            firstColumnIndex = leftMapping(firstColumnIndex, bw, first, tallyFactor, tally)
            difference += 1
        return (sa[firstColumnIndex // saFactor] + difference) % len(bw)

In [ ]:
assert position(4,[6, 2, 0, 1],2,'abba$aa',{'$': (0,1), 'a': (1,5), 'b': (5,7)}, 2, {'a': [1, 1, 2, 4], 'b': [0, 2, 2, 2], '$': [0, 0, 1, 1]}) == 0

In [ ]:
def leftMapping(index, bw, first, tallyFactor ,tally):
        c = bw[index]
        bRank = getBRank(index, bw, tallyFactor, tally)
        return first[c][0] + bRank

In [ ]:
assert leftMapping(3,'abba$aa',{'$': (0,1), 'a': (1,5), 'b': (5,7)}, 1, {'a': [1, 1, 1, 2, 2, 3, 4], 'b': [0, 1, 2, 2, 2, 2, 2], '$': [0, 0, 0, 0, 1, 1, 1]}) == 2

In [ ]:
def getBRank(index, bw, tallyFactor, tally):
        c = bw[index]
        if c == '$':
            return 0
        return getTallyRank(c, index, tallyFactor, tally, bw) - 1


In [ ]:
assert getBRank(3, 'abba$aa', 1, {'a': [1, 1, 1, 2, 2, 3, 4], 'b': [0, 1, 2, 2, 2, 2, 2], '$': [0, 0, 0, 0, 1, 1, 1]}) == 1

In [ ]:
  def getTallyRank(c, index, tallyFactor, tally, bw):
        tallyIndex = index // tallyFactor
        count = tally[c][tallyIndex]
        currentIndex = tallyIndex * tallyFactor + 1
        while (currentIndex <= index):
            if bw[currentIndex] == c:
                count += 1
            currentIndex += 1
        return count

In [ ]:
assert getTallyRank('b', 4,2, {'a': [1, 1, 2, 4], 'b': [0, 2, 2, 2], '$': [0, 0, 1, 1]},'abba$aa') == 2


In [43]:
pip install bio

Note: you may need to restart the kernel to use updated packages.


In [44]:
pip install pympler

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import sys
import psutil
import time
import pandas as pd
from Bio import SeqIO
from pympler import asizeof 
def readSequence(file_path):
    with open(file_path) as file:
        for record in SeqIO.parse(file, "fasta"):
            return str(record.seq)


file_directory_path = '/sbgenomics/project-files/'
df_row_list = []
testData = {
    'files': [
        {
            'name': 'Coffea arabica',
            'file': 'Coffea arabica chromosome 1c.fasta',
            'patterns': [ 'ATGCATG', 'TCTCTCTA', 'TTCACTACTCTCA']
        },
        
        {
            'name': 'Canis lupus familiaris',
            'file': 'Canis lupus familiaris breed boxer chromosome 3.fasta',
            'patterns': [ 'GGAGTC', 'CAGCCCCACGGA', 'AGCGCC']
        },
        
        {
            'name': 'Mus pahari',
            'file': 'Mus pahari chromosome X.fasta',
            'patterns': [ 'ATGATG', 'CTCTCTA', 'TCACTACTCTCA']
        }
    ],
    'sa_factors': [ 4, 16, 64, 256 ],
    'tally_factors': [ 8, 32, 128, 512 ]
}
for file in testData['files']:
    print("Start analizing file ", file['name'])
    text = readSequence(file_directory_path + file['file'])
    saSimple = makeSuffixArray(str.encode(text))
    suffix_array_memory = asizeof.asizeof(saSimple)
    bw = bwt(text + '$', saSimple)
    l_memory = asizeof.asizeof(bw)
    ranks, tots = rankBwt(bw)
    rank_memory = asizeof.asizeof(ranks)
    first = firstCol(tots)
    f_memory = asizeof.asizeof(first)
    for pattern in file['patterns']:
     print("Start searching for", pattern)
     start1 = time.time()
     find = indexSearch(bw, pattern, saSimple, ranks, tots)
     end1 = time.time()
     print((l_memory + f_memory + rank_memory + suffix_array_memory)/ 1024 / 1024)
     print(pattern, "found on", len(find) if find != None else 0, "positions in", end1 - start1, "seconds using  simple algorithm")
     df_row = {
                        'filename': file['name'],
                        'pattern': pattern,
                        'occurrences': len(find),
                        'sa_factor': None,
                        'tally_factor': None,
                        'time': end1 - start1,
                        'mem':  (l_memory + f_memory + rank_memory + suffix_array_memory)/ 1024 / 1024
                }
                                   
     df_row_list.append(df_row)
     for sa_factor in testData['sa_factors']:
       sa1 = downsampleSuffixArray(saSimple, sa_factor)
       suffix_array_memory = asizeof.asizeof(sa1)
       #bw = bwt(text[0:1000000] + '$', saSimple)
       #l_memory = asizeof.asizeof(bw)
       for tally_factor in testData['tally_factors']:
         
         rankAll, tots = rankAllBwt(bw, tally_factor)
         tally_rank_memory = asizeof.asizeof(rankAll)
         first = firstCol(tots)
         f_memory = asizeof.asizeof(first)
         start2 = time.time()
         findOpt = indexSearchOptimized(bw,pattern, sa1, sa_factor, tally_factor, rankAll, tots)
         end2 = time.time()
         print((l_memory + f_memory + tally_rank_memory + suffix_array_memory)/ 1024 / 1024)
         print(pattern, "found on", len(findOpt) if findOpt != None else 0, "positions in", end2 - start2, "seconds using optimized algorithm" , [sa_factor, tally_factor])
         df_row = {
                        'filename': file['name'],
                        'pattern': pattern,
                        'occurrences': len(findOpt),
                        'sa_factor': sa_factor,
                        'tally_factor': tally_factor,
                        'time': end2 - start2,
                        'mem': (l_memory + f_memory + tally_rank_memory + suffix_array_memory)/ 1024 / 1024
                    }
                                   
         df_row_list.append(df_row)


    df = pd.DataFrame(df_row_list)
    df.to_csv(file['name'] + 'output.csv', index=None)
    df_row_list = []

Start analizing file  Coffea arabica
Start searching for ATGCATG
3929.6301193237305
ATGCATG found on 6529 positions in 0.7879695892333984 seconds using  simple algorithm
1374.880760192871
ATGCATG found on 6529 positions in 0.4021303653717041 seconds using optimized algorithm [4, 8]
796.2585220336914
ATGCATG found on 6529 positions in 0.4227163791656494 seconds using optimized algorithm [4, 32]
600.3952407836914
ATGCATG found on 6529 positions in 0.4993274211883545 seconds using optimized algorithm [4, 128]
549.7729110717773
ATGCATG found on 6529 positions in 0.8075652122497559 seconds using optimized algorithm [4, 512]
1013.6677322387695
ATGCATG found on 6529 positions in 0.25476837158203125 seconds using optimized algorithm [16, 8]
435.04549407958984
ATGCATG found on 6529 positions in 0.36244916915893555 seconds using optimized algorithm [16, 32]
239.18221282958984
ATGCATG found on 6529 positions in 0.7340028285980225 seconds using optimized algorithm [16, 128]
188.55988311767578
ATGC